In [2]:
import transform 
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils

In [58]:
image = cv2.imread('../../images/test1.jpeg')

In [59]:
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image,height = 500)
print(image.shape)

(500, 375, 3)


In [65]:
def auto_canny(image, sigma=0.33):
	# compute the median of the single channel pixel intensities
	v = np.median(image)
 
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
 
	# return the edged image
	return edged

In [66]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0)
ret,thr = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
# cv2.imshow('win1', thr)
# cv2.waitKey(0)  
# cv2.destroyAllWindows()
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
edged = auto_canny(blurred)

In [67]:
cv2.imshow('Edged',edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [63]:
def extract_corners(image):
        """
        Find the 4 corners of a binary image
        :param image: binary image
        :return: 4 main vertices or None
        """
        cnts, _ = cv2.findContours(image.copy(),
                                   cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)
        
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        _, _, h, w = cv2.boundingRect(cnt)
        epsilon = min(h, w) * 0.5
        vertices = cv2.approxPolyDP(cnt, epsilon, True)
        vertices = cv2.convexHull(vertices, clockwise=True)
        
        return vertices

In [64]:
vertices = extract_corners(edged)
cp = cv2.drawContours(image,[vertices],-1,(255,0,0), 3)
cv2.imshow('image',cp)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [70]:
vertices = vertices.reshape(4,2) * ratio
trans = transform.Transform()
warped = trans.four_point_transform(orig,vertices)

ValueError: cannot reshape array of size 10 into shape (4,2)